# Time Frequency analysis in ECoG

This week's lab will explore the frequency representation of ECoG. We'll calculate the full time-frequency representation of an ECoG signal, and then focus on a particular frequency band of interest.

We'll now be going back into the world of ECoG, to see what differences we find. We'll use the same "consonant/dissonant" dataset seen in previous weeks. Perhaps by investigating the time-frequency representation of the signal we will uncover things that we couldn't see before...

In [ ]:
import mne
import numpy as np
import datascience as ds
import neurods as nds
import matplotlib.pyplot as plt
%matplotlib widget

* Load in the raw data from the consonant/dissonant task
* Also load the image of the brain and layout of ECoG channels
* Finally, load in the event timing information for this task.

In [ ]:
path_ecog = '../../data/ecog/chords_task/'
data_path = path_ecog + 'ecog_resamp-raw.fif'
event_path = path_ecog + 'meta_time.csv'
brain_image_path = path_ecog + 'brain.png'
layout_path = path_ecog + 'brain.lout'

### STUDENT ANSWER


* Convert the events information into an MNE events object. Make sure to include all event types in the events object.
* Also create an events dictionary that maps event types onto event ID numbers.

In [ ]:
### STUDENT ANSWER


* Using this events array and our events dictionary, convert the `Raw` data to an `Epochs` object
* Plot the global field power for each event type.

In [ ]:
### STUDENT ANSWER


In [ ]:
### STUDENT ANSWER


This is a plot we generated last week. Let's see if something else comes out of a time-frequency analysis.

* Convert the `Epochs` object into a time-frequency representation.
  * Use frequencies from 5 to 140 spaced 2Hz apart
  * Accomplish this with mne `tfr_array_morlet` function.
  * Print the shape of the output for each condition. What does each axis represent?

In [ ]:
### STUDENT ANSWER


* For each condition:
  * Convert the output of `tfr_array_morlet` into an `AverageTFR` object. This will let you plot it. Use the docstring from `AverageTFR` to figure out how to create an object from data.
  * Plot a topographic map of the TFR for each electrode. Use the `plot_tfr_topo` function in neurods.

In [ ]:
### STUDENT ANSWER


* Finally, plot the difference (dissonant - consonant) for each TFR. Convert this into an MNE `AverageTFR` object.
* Plot the result on the brain as well.

In [ ]:
### STUDENT ANSWER


* Do you see any differences between the two groups?
* How would you test for a difference between the two?

# Inspecting high-frequency activity
It seems like there are two "blobs" of high-frequency activity in the bove plots. Moreover, these blobs are more localized to specific electrodes than the ERPs of raw activity we saw before. Let's take a look at this activity.

* Extract the first Epoch from the consonant data. Insert it into a new variable.
* Using a bank of morlet wavelets, filter the epoch with 5 equally-spaced frequencies from 70-140Hz.
* Take the "real" component of the result (using `np.real`), and average across frequencies.

In [ ]:
### STUDENT ANSWER


* Plot the result in a single plot

In [ ]:
### STUDENT ANSWER


* What do you notice about the edges of the plot?
* Try the same process with a few other epochs, see if the edges look consistent.

The peaks that you see at the corners are called "edge artifacts". In order to get around them, we need to remove timepoints near the edges of our data (e.g., at the beginning and end). As such, it is generally better to extract the frequency amplitudes from the **`Raw`** data, and then convert it to epochs.

* Using the `extract_amplitude` function described below, extract 5 linearly-spaced amplitudes from 70-150 Hz (define them manually or try using `np.linspace`). This function does the same thing that you did above, but it is more efficient with memory use.
* Turn this `Raw` object into an `Epochs` object using the same event times that we used above.

In [ ]:
from tqdm import tqdm
import warnings

def extract_amplitude(inst, freqs, n_cycles=7, normalize=False):
    """Extract the time-varying amplitude for a frequency band.

    If multiple freqs are given, the amplitude is calculated at each frequency
    and then averaged across frequencies.

    Parameters
    ----------
    inst : instance of Raw or Epochs
        The data to have amplitude extracted.
    freqs : array of ints/floats, shape (n_freqs)
        The frequencies to use. If multiple frequencies are given, amplitude
        will be extracted at each and then averaged between frequencies.
    n_cycles : int
        The number of cycles to include in the filter length for the wavelet.
    normalize : bool
        Normalize the power of each band by its mean before combining.

    Returns
    -------
    inst : mne instance, same type as input 'inst'
        The MNE instance with channels replaced with their time-varying
        amplitude for the supplied frequency range.
    """

    # Data checks
    freqs = np.atleast_1d(freqs)
    if not inst.preload:
        raise ValueError('Data must be preloaded.')

    data = inst.get_data()
    data=data[np.newaxis, :]
    sfreq = inst.info['sfreq']
    n_channels, n_times = data.shape[1], data.shape[2]

    # Initialize array to store the band amplitudes
    bands = np.zeros((1, n_channels, n_times))

    for ifreq in tqdm(freqs, desc="Processing frequencies"):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # Perform time-frequency decomposition
            tfr_complex = mne.time_frequency.tfr_array_morlet(
                data, sfreq=sfreq, freqs=[ifreq], n_cycles=n_cycles, output='complex'
            )
        
        # Compute the amplitude (absolute value of the complex TFR)
        band = np.abs(tfr_complex[:, :, 0, :])
        
        if normalize:
            band /= band.mean(axis=-1, keepdims=True)

        bands += band

    # Average across frequencies
    bands /= len(freqs)

    # Create a new MNE object with the amplitude data
    inst_amplitude = inst.copy()
    inst_amplitude._data = bands.reshape(n_channels,n_times)

    return inst_amplitude

In [ ]:
### STUDENT ANSWER


* Create an average response for each event type 
* Plot this average response (with the `.plot` method)
* For make a list of the top 3 active channels that you determine by visual inspection (you can see the channel names by clicking on each trace)

In [ ]:
### STUDENT ANSWER


In [ ]:
### STUDENT ANSWER


* Now, plot each one on the brain using the ECoG layout we loaded above. Remember we can do this with the `plot_topo` function, and don't forget to include an image of the brain as well (which we've also already loaded).

In [ ]:
### STUDENT ANSWER


* Finally, plot the difference (dissonant - consonant) between the two on the brain. You can calculate the difference using mne.combine_evoked().


In [ ]:
### STUDENT ANSWER


* Does it look like there is a difference between the two?
* Is it more or less localized than the signals that we've looked at so far?
* Why or why not?

In [ ]:
### STUDENT ANSWER


# Bonus: Confidence intervals
Now that we've calculated the evoked high-frequency amplitude for two conditions, let's test to see if there is a difference between them.

* For the 3 most active electrodes:
* Using the bootstrap method, calculate the confidence interval for the difference between our two conditions
  * Note that our trials are not naturally paired, so you'll have to take a sample of trials in each group first, calculate their mean, and then store that value
* Plot the histogram of bootstrapped differences, as well as 95 and 99% confidence intervals.
* Do you conclude that there is a significant difference between the two conditions for any of these channels?
* Why do you think this is, or is not, the case?

In [ ]:
### STUDENT ANSWER
